In [256]:
import pandas as pd
import numpy as np
import time
import datetime
import os
import pickle
from Bio import Entrez
import requests
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from collections import defaultdict
from ast import literal_eval

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

### Extract pubmed ID

In [2]:
Entrez.email='xieyc10@gmail.com'

In [230]:
base_dir='/Volumes/Yuchen_Drive/Insight/OncoMatch'
data_df=pickle.load(open(os.path.join(base_dir, "data/Oncologist_info_1470.pkl"),"rb"))

In [4]:
data_df.shape

(1470, 9)

In [236]:
num_list=[]
pub_dict=defaultdict(list)

In [237]:
for name in data_df.name:
    if name not in pub_dict:
        search_term='cancer[MESH] AND {}[AUTH]'.format(name)
        search_handle=Entrez.esearch(db='Pubmed',term=search_term,retmax=1000,idtype="acc")
        search_results = Entrez.read(search_handle)
        search_handle.close()
        num_list.append(int(search_results['Count']))
        pub_dict[name]=search_results['IdList']
        time.sleep(0.5)

In [238]:
len(num_list)

1470

In [39]:
data_df['article_num']=num_list

In [45]:
data_df.head(10)

,name,degree,phone,center_name2,address,city_state,speciality,certificate,center_name,article_num
0,John H. Glick,"MD, FASCO",(215) 662-6334,University of Pennsylvania-Abramson Cancer Center,"3400 Civic Center Blvd3rd Fl, Ste 3-300S","Philadelphia, PA 19104-5127, US","[Breast Cancer, Cancer Prevention]","[Internal Medicine, Medical Oncology]",Abramson Cancer Center,7
1,Arthur M. Feldman,MD,(215) 662-9801,University of Pennsylvania-Abramson Cancer Center,Penn Presbyterian Medcl Ctr51 N 39th St MAB St...,"Philadelphia, PA 19104, US","[Breast Cancer, Geriatrics Oncology]","[Internal Medicine, Medical Oncology]",Abramson Cancer Center,0
2,David M. Mintzer,MD,Search for Phone Number,Abramson Cancer Center at Pennsylvania Hospital,230 W Washington Sq Fl 2,"Philadelphia, PA 19106-3500, US","[Breast Cancer, Lung Cancer, Palliative Care/E...","[Hematology, Hospice and Palliative Medicine, ...",Abramson Cancer Center,13
3,David J. Vaughn,MD,(215) 349-8140,University of Pennsylvania-Abramson Cancer Center,PCAM South 10-1143400 Civic Center Blvd,"Philadelphia, PA 19104, US","[Bladder Cancer, Prostate Cancer, Testicular C...",[Medical Oncology],Abramson Cancer Center,86
4,Charles John Schneider,"MD, FACP",Search for Phone Number,"Hospital of the University of Pennsylvania, Ab...",Pereleman Center for Advanced Medicine3400 Civ...,"Philadelphia, PA 19104, US","[Clinical Research, Developmental Therapeutics...",[Medical Oncology],Abramson Cancer Center,0
5,Marcia S. Brose,"MD, PhD, FASCO",(215) 615-0741,University of Pennsylvania-Abramson Cancer Center,5 Silverstein3400 Spruce St,"Philadelphia, PA 19104, US","[Head and Neck Cancer, Clinical Research, Clin...","[Internal Medicine, Medical Oncology]",Abramson Cancer Center,50
6,Robert G. Maki,"MD, PhD, FACP, FASCO",800-789-7366,Abramson Cancer Center,Perelman Center for Advanced Medicine3400 Civi...,"Philadelphia, Pennsylvania 19104","[Bone Cancer, Sarcoma, Clinical Trials/Biostat...","[Internal Medicine, Medical Oncology]",Abramson Cancer Center,162
7,Cheryl A. Johnson,MD,(610) 738-2500,Abramson Cancer Center at Chester County,440 E Marshall St Ste 201,"West Chester, Pennsylvania 19380-5414, US","[Breast Cancer, Lung Cancer]","[Hematology, Medical Oncology]",Abramson Cancer Center,0
8,Jonathan D. Abramson,MD,Search for Phone Number,"Jonathan D. Abramson, MD",3140 W Campus Dr,"Bay City, MI 48706-2776, US",[],[Hematology],Abramson Cancer Center,0
9,Lee P. Hartner,MD,(215) 829-6088,University of Pennsylvania-Abramson Cancer Center,230 W Washington Sq Fl 2U of Penn Abramson Can...,"Philadelphia, PA 19106-3500, US","[Head and Neck Cancer, Sarcoma, Palliative Car...","[Hematology, Hospice and Palliative Medicine, ...",Abramson Cancer Center,1


In [47]:
pickle.dump(data_df, open(os.path.join(base_dir, "data/Oncologist_info_1470.pkl"), "wb"))
data_df.to_csv(os.path.join(base_dir, "data/Oncologist_info_1470.csv"), index=False)
pickle.dump(pub_dict, open(os.path.join(base_dir, "data/Oncologist_info_1470_pub_dict.pkl"), "wb"))

In [245]:
pickle.dump(pub_dict, open(os.path.join(base_dir, "data/Oncologist_info_1470_pub_dict.pkl"), "wb"))

In [246]:
pub_dict=dict(pub_dict)

In [247]:
type(pub_dict)

dict

In [222]:
to_del=[]
for i in pub_dict:
    if len(pub_dict[i])==0:
        to_del.append(i)

In [224]:
for i in to_del:
    del pub_dict[i]

In [226]:
print('Number of doctors with publications: {}'.format(len(pub_dict)))

Number of doctors with publications: 1153


In [233]:
pickle.dump(pub_dict, open(os.path.join(base_dir, "data/doctor2pmid_1153_dict.pkl"), "wb"))

In [234]:
doctor2pmid_dict=pickle.load(open(os.path.join(base_dir, "data/doctor2pmid_1153_dict.pkl"),"rb"))

In [259]:
doctor2pmid_1153_dict_df = pd.DataFrame.from_dict(doctor2pmid_dict, orient='index')
doctor2pmid_1153_dict_df['pmid'] = doctor2pmid_1153_dict_df.apply(lambda x:[int(i) for i in x if i is not None],axis=1)
doctor2pmid_1153_dict_df = doctor2pmid_1153_dict_df.loc[:,'pmid'].to_frame()
doctor2pmid_1153_dict_df.head()

,pmid
John H. Glick,"[19863338, 18621616, 18591552, 15210738, 12712..."
David M. Mintzer,"[23233720, 22154117, 20053760, 18758307, 17573..."
David J. Vaughn,"[29848605, 29617189, 29590008, 29534941, 29523..."
Marcia S. Brose,"[30832606, 29704768, 29657135, 29656442, 29466..."
Robert G. Maki,"[31208434, 31142482, 30664786, 30575484, 30383..."


In [260]:
doctor2pmid_1153_dict_df.to_csv(os.path.join(base_dir, 'data/doctor2pmid_1153_dict_df.csv'),index=True,index_label='name')

In [265]:
doctor2pmid_1153_dict_df=pd.read_csv(os.path.join(base_dir, 'data/doctor2pmid_1153_dict_df.csv'),converters={"pmid":literal_eval})

In [266]:
doctor2pmid_1153_dict_df.head()

,name,pmid
0,John H. Glick,"[19863338, 18621616, 18591552, 15210738, 12712..."
1,David M. Mintzer,"[23233720, 22154117, 20053760, 18758307, 17573..."
2,David J. Vaughn,"[29848605, 29617189, 29590008, 29534941, 29523..."
3,Marcia S. Brose,"[30832606, 29704768, 29657135, 29656442, 29466..."
4,Robert G. Maki,"[31208434, 31142482, 30664786, 30575484, 30383..."


In [248]:
len(doctor2pmid_dict)

1153

In [170]:
data_df=pickle.load(open(os.path.join(base_dir, "data/Oncologist_info_1470.pkl"),"rb"))

In [49]:
print('Total number of articles: {}'.format(data_df.article_num.sum()))

Total number of articles: 91145


### Extract pubmed Abstract

In [51]:
pub_dict=pickle.load(open(os.path.join(base_dir, "data/Oncologist_info_1470_pub_dict.pkl"), "rb"))

In [52]:
pub_list=[]
for i in pub_dict.values():
    pub_list+=i

In [53]:
pub_array = np.unique(np.array(pub_list))

In [54]:
print('Total number of unique articles: {}'.format(len(pub_array)))

Total number of unique articles: 57605


### Extract pubmed Abstract_combined dataframe

In [55]:
articletitle_dict = {}
journaltitle_dict = {}
journalabbre_dict = {}
articledate_dict = {}
abstract_dict = {}
articletype_dict = {}

In [56]:
len(np.unique(np.array(pub_list)))

57605

In [57]:
pub_array = np.unique(np.array(pub_list))

In [58]:
pub_list=list(pub_array)
print(len(pub_list))

57605


In [59]:
for i in range(6):
    fetch_handle=Entrez.efetch(db='Pubmed',id=pub_list[i*10000:(i+1)*10000],retmode='xml',rettype='medline')
    fetch_results = Entrez.read(fetch_handle)
    fetch_handle.close()
    for item in fetch_results['PubmedArticle']:
        pmid = item['MedlineCitation']['PMID']
        article = item['MedlineCitation']['Article']
        articletitle_dict[pmid]=article['ArticleTitle']
        journaltitle_dict[pmid]=article['Journal']['Title']
        if 'ISOAbbreviation' in article['Journal']:
            journalabbre_dict[pmid]=article['Journal']['ISOAbbreviation']
    
        if len(article['ArticleDate']) > 0:
            article_date=article['ArticleDate'][0]
            articledate_dict[pmid] = datetime.date(int(article_date['Year']),int(article_date['Month']),int(article_date['Day']))
        elif 'Year' in article['Journal']['JournalIssue']['PubDate']:
            pubdate=article['Journal']['JournalIssue']['PubDate']
            if 'Month' in pubdate:
                if pubdate['Month'].isdigit():
                    date_str=pubdate['Year']+", "+pubdate['Month']
                    articledate_dict[pmid] = datetime.datetime.strptime(date_str, "%Y, %m").date()
                else:
                    date_str=pubdate['Year']+", "+pubdate['Month']
                    articledate_dict[pmid] = datetime.datetime.strptime(date_str, "%Y, %b").date()
            else:
                date_str=pubdate['Year']+", "+'Jan'
                articledate_dict[pmid] = datetime.datetime.strptime(date_str, "%Y, %b").date()
    
        if 'Abstract' in article:
            abstract_dict[pmid]=" ".join(article['Abstract']['AbstractText'])  
        articletype_dict[pmid]=str(article['PublicationTypeList'][0])

In [60]:
df1 = pd.DataFrame.from_dict(articletitle_dict, orient='index', columns=['article_title'])
df2 = pd.DataFrame.from_dict(journaltitle_dict, orient='index', columns=['journal_title'])
df3 = pd.DataFrame.from_dict(journalabbre_dict, orient='index', columns=['journal_abbre'])
df4 = pd.DataFrame.from_dict(articledate_dict, orient='index', columns=['article_date'])
df5 = pd.DataFrame.from_dict(abstract_dict, orient='index', columns=['abstract'])
df6 = pd.DataFrame.from_dict(articletype_dict, orient='index', columns=['article_type'])

In [61]:
abstract_df=pd.concat([df1,df2,df3,df4,df5,df6],axis=1,sort=False)

In [63]:
print(abstract_df.shape)

(57605, 6)


In [64]:
abstract_df.index.nunique()

57605

In [65]:
today = datetime.date.today()

In [66]:
abstract_df['pub_days']=abstract_df['article_date'].apply(lambda x:(today-x).days if type(x)==datetime.date else np.nan)

In [67]:
abstract_df.head()

,article_title,journal_title,journal_abbre,article_date,abstract,article_type,pub_days
10037348,In vivo trial comparing patients' tolerance of...,Lasers in surgery and medicine,Lasers Surg Med,1999-01-01,Different types of Q-switched lasers have been...,Clinical Trial,7690.0
10079468,Chronic myelogenous leukemia: update on biolog...,"Oncology (Williston Park, N.Y.)","Oncology (Williston Park, N.Y.)",1999-02-01,Chronic myelogenous leukemia (CML) is a myelop...,Comparative Study,7659.0
10581602,Pain management in patients with advanced pros...,"Oncology (Williston Park, N.Y.)","Oncology (Williston Park, N.Y.)",1999-11-01,Prostate cancer is the most commonly diagnosed...,Journal Article,7386.0
10873802,A human homolog of the C. elegans polarity det...,Current biology : CB,Curr. Biol.,2000-06-01,Rac and Cdc42 are members of the Rho family of...,Journal Article,7173.0
11050493,An in vivo trial comparing the clinical effica...,Dermatologic surgery : official publication fo...,Dermatol Surg,2000-10-01,Q-switched 755 nm alexandrite (QS alex) and Q-...,Clinical Trial,7051.0


In [68]:
abstract_df.isnull().sum()

article_title       0
journal_title       0
journal_abbre       4
article_date     1090
abstract         6622
article_type        0
pub_days         1090
dtype: int64

In [96]:
abstract_df.notnull().sum()

article_title    57605
journal_title    57605
journal_abbre    57601
article_date     56515
abstract         50983
article_type     57605
pub_days         56515
dtype: int64

In [70]:
abstract_df.describe(include='all')

,article_title,journal_title,journal_abbre,article_date,abstract,article_type,pub_days
count,57605,57605,57601,56515,50983,57605,56515.000000
unique,57304,2011,2009,5108,50911,41,NaN
top,Editorial comment.,Journal of clinical oncology : official journa...,J. Clin. Oncol.,2005-01-01,,Journal Article,NaN
freq,30,3748,3748,226,53,39401,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2886.088472
std,NaN,NaN,NaN,NaN,NaN,NaN,1727.427447
min,NaN,NaN,NaN,NaN,NaN,NaN,-11.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,1421.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,2646.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,4221.000000


In [71]:
pickle.dump(abstract_df, open(os.path.join(base_dir, "data/abstract_df_57605.pkl"), "wb"))

In [133]:
abstract_df=pickle.load(open(os.path.join(base_dir, "data/abstract_df_57605.pkl"),"rb"))

In [134]:
abstract_df.shape

(57605, 7)

In [76]:
abstract_df.to_csv(os.path.join(base_dir, 'data/abstract_df_57605.csv'),index_label='pmid')

In [77]:
abstract_df=pd.read_csv(os.path.join(base_dir, 'data/abstract_df_57605.csv'),index_col='pmid')

In [79]:
abstract_df.describe(include='all')

,article_title,journal_title,journal_abbre,article_date,abstract,article_type,pub_days
count,57605,57605,57601,56515,50930,57605,56515.000000
unique,57304,2011,2009,5108,50910,41,NaN
top,Editorial comment.,Journal of clinical oncology : official journa...,J. Clin. Oncol.,2005-01-01,The benefit of salvage chemotherapy is modest ...,Journal Article,NaN
freq,30,3748,3748,226,2,39401,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2886.088472
std,NaN,NaN,NaN,NaN,NaN,NaN,1727.427447
min,NaN,NaN,NaN,NaN,NaN,NaN,-11.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,1421.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,2646.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,4221.000000


In [80]:
abstract_df.head()

,article_title,journal_title,journal_abbre,article_date,abstract,article_type,pub_days
pmid,,,,,,,
10037348,In vivo trial comparing patients' tolerance of...,Lasers in surgery and medicine,Lasers Surg Med,1999-01-01,Different types of Q-switched lasers have been...,Clinical Trial,7690.0
10079468,Chronic myelogenous leukemia: update on biolog...,"Oncology (Williston Park, N.Y.)","Oncology (Williston Park, N.Y.)",1999-02-01,Chronic myelogenous leukemia (CML) is a myelop...,Comparative Study,7659.0
10581602,Pain management in patients with advanced pros...,"Oncology (Williston Park, N.Y.)","Oncology (Williston Park, N.Y.)",1999-11-01,Prostate cancer is the most commonly diagnosed...,Journal Article,7386.0
10873802,A human homolog of the C. elegans polarity det...,Current biology : CB,Curr. Biol.,2000-06-01,Rac and Cdc42 are members of the Rho family of...,Journal Article,7173.0
11050493,An in vivo trial comparing the clinical effica...,Dermatologic surgery : official publication fo...,Dermatol Surg,2000-10-01,Q-switched 755 nm alexandrite (QS alex) and Q-...,Clinical Trial,7051.0


### Extract citation information

In [135]:
abstract_df2=abstract_df.dropna(subset=['abstract'])

In [136]:
abstract_df2.shape

(50983, 7)

In [126]:
abstract_df=abstract_df[-abstract_df.abstract.isnull()]

In [127]:
abstract_df.shape

(50983, 7)

In [105]:
abstract_df.to_csv(os.path.join(base_dir, 'data/abstract_df_50930.csv'),index=True,index_label='pmid')

In [137]:
abstract_df=pd.read_csv(os.path.join(base_dir, 'data/abstract_df_50930.csv'),index_col='pmid')

In [138]:
abstract_df.head()

,article_title,journal_title,journal_abbre,article_date,abstract,article_type,pub_days
pmid,,,,,,,
10037348,In vivo trial comparing patients' tolerance of...,Lasers in surgery and medicine,Lasers Surg Med,1999-01-01,Different types of Q-switched lasers have been...,Clinical Trial,7690.0
10079468,Chronic myelogenous leukemia: update on biolog...,"Oncology (Williston Park, N.Y.)","Oncology (Williston Park, N.Y.)",1999-02-01,Chronic myelogenous leukemia (CML) is a myelop...,Comparative Study,7659.0
10581602,Pain management in patients with advanced pros...,"Oncology (Williston Park, N.Y.)","Oncology (Williston Park, N.Y.)",1999-11-01,Prostate cancer is the most commonly diagnosed...,Journal Article,7386.0
10873802,A human homolog of the C. elegans polarity det...,Current biology : CB,Curr. Biol.,2000-06-01,Rac and Cdc42 are members of the Rho family of...,Journal Article,7173.0
11050493,An in vivo trial comparing the clinical effica...,Dermatologic surgery : official publication fo...,Dermatol Surg,2000-10-01,Q-switched 755 nm alexandrite (QS alex) and Q-...,Clinical Trial,7051.0


In [144]:
abstract_df.shape

(50930, 7)

### Get citations

In [500]:
with open('data/apikey.txt', 'r') as file:
    apikey = file.read().strip('\n')

In [501]:
apikey

'9bd17f6cbbc39d567daf1b46c4fa1c24'

In [111]:
apikey='c348e321a28903840035023148ca6e95'

In [140]:
base_api = 'https://api.elsevier.com/content/search/scopus'

In [145]:
citations = {}

In [147]:
apikey='6d0bf6749ccb91c6b8c6979349441ee9'

In [157]:
apikey='b9f56accd684e10a904f6123be409e71'

In [158]:
for pmid in abstract_df.index:
    if pmid not in citations:
        query_pmid = 'PMID(' + str(pmid) + ')'
        params = {
        'apiKey': apikey,
        'query': query_pmid,
        'field': 'citedby-count'
    }
        response = requests.get(base_api, params=params)
        response = json.loads(response.text)
        if 'search-results' in response:
            if 'citedby-count' in response['search-results']['entry'][0]:
                citations[pmid]=int(response['search-results']['entry'][0]['citedby-count'])

In [159]:
len(citations)

49665

In [160]:
citation_df=pd.DataFrame.from_dict(citations,orient='index')
citation_df.columns=['citation']
citation_df.head()

,citation
10037348,59
10079468,73
10581602,25
10873802,219
11050493,55


In [161]:
print('{} abstracts have citations'.format(len(citation_df)))

49665 abstracts have citations


In [162]:
abstract_citation_df=pd.merge(abstract_df, citation_df, left_index=True, right_index=True, how='outer')
abstract_citation_df.head()

,article_title,journal_title,journal_abbre,article_date,abstract,article_type,pub_days,citation
2357786,Pseudohypoproteinemia and multiple myeloma.,Cleveland Clinic journal of medicine,Cleve Clin J Med,1990-05-01,Paraproteinemia is an important diagnostic fea...,Case Reports,10857.0,1.0
7512659,Comparison of digital rectal examination and s...,The Journal of urology,J. Urol.,1994-05-01,To compare the efficacy of digital rectal exam...,Clinical Trial,9396.0,1218.0
7523707,Accuracy of digital rectal examination and tra...,The Journal of urology,J. Urol.,1994-11-01,Not all prostate cancers are sonographically h...,Clinical Trial,9212.0,116.0
7525995,Selection of optimal prostate specific antigen...,The Journal of urology,J. Urol.,1994-12-01,A prospective clinical trial of prostate cance...,Clinical Trial,9182.0,230.0
7692657,Effect of patient age on early detection of pr...,Urology,Urology,1993-10-01,This study was designed to determine the effec...,Clinical Trial,9608.0,132.0


In [163]:
abstract_citation_df.shape

(50930, 8)

In [164]:
abstract_citation_df.to_csv(os.path.join(base_dir, 'data/abstract_citation_df_50930.csv'),index=True,index_label='pmid')

In [165]:
abstract_citation_df=pd.read_csv(os.path.join(base_dir, 'data/abstract_citation_df_50930.csv'),index_col='pmid')

In [166]:
abstract_citation_df.shape

(50930, 8)